# Xgboost

Use Xgboost as model

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
feature_files = [
    'Abhishek_features',
    'magic_feature',
    'magic_feature_v1',
#     'lystdo_correctwords_lstm',
    'Howard_feature',
#     'lystdo_1234_loss017',
    'HubertLin_features_raw',
    'HubertLin_features_simple_tokenizer',
    'HubertLin_features_word_corrected',
    'pagerank',
    'word_match_share',
    'fasttext_distance',
#     'modified_count',
    'magic_v25_qid',
    'k_scrore',
    'pos_dist',
    'dep_dist',
]


In [3]:
take_feature = {
    'lystdo_origin_loss018': [
        'lystdo_original_c',
#         'lystdo_original_p',
#         'lystdo_original_k',
#         'lystdo_original_b'
    ]
}

def read_csv(file_name, base=''):
    df = pd.read_csv(base + file_name + '.csv')
    if file_name in take_feature:
        df = df[take_feature[file_name]]
    return df
        

## Merge all features

In [4]:
data_frames = [read_csv(file, base='features_from_model/train/') for file in feature_files]
df_train = pd.concat(data_frames, axis=1)

In [5]:
y_train = pd.read_csv('../dataset/quora-question-pairs/train.csv')['is_duplicate']

## Get original val set idxes

In [6]:
# %%time

# val_pos_pairs = pickle.load(open('../dataset/augmented/validation_positive_qid_pairs.pkl','rb'))
# val_neg_pairs = pickle.load(open('../dataset/augmented/validation_negative_qid_pairs.pkl','rb'))

# df = pd.read_csv('../dataset/quora-question-pairs/train.csv')

# def get_idx(qid_pair, label):
#     res = df[np.bitwise_and(df['qid1']==qid_pair[0],df['qid2']==qid_pair[1])]
    
#     # since we do augmentation in val set also, there might be some samples not exist in original train set
#     if len(res)==0:
#         return np.nan
#     elif res['is_duplicate'].tolist()[0]!=label:
#         return np.nan
#     return res.index[0]

# pos_idxes = np.unique([get_idx(pair, 1) for pair in val_pos_pairs])
# neg_idxes = np.unique([get_idx(pair, 0) for pair in val_neg_pairs])

# # # workaround
# pos_idxes = [idx for idx in pos_idxes if not np.isnan(idx)]
# neg_idxes = [idx for idx in neg_idxes if not np.isnan(idx)]

# all_idxes = np.concatenate([pos_idxes,neg_idxes])

# del df
# del val_pos_pairs
# del val_neg_pairs

# val_idxes = np.concatenate([pos_idxes, neg_idxes]).astype('int')
# y_val = np.concatenate([np.ones(len(pos_idxes)),np.zeros(len(neg_idxes))]).astype('int')

# pickle.dump([val_idxes, y_val], open('./val/val_data.pkl','wb'))
# pickle.dump(all_idxes, open('./val/val_idxes.pkl','wb'))


In [7]:
val_idxes, y_val = pickle.load(open('./val/val_data.pkl','rb'))

In [8]:
def inf_nan_to_zero(arr):
    nan = np.isnan(arr)
    inf = np.isinf(arr)
    arr[nan] = 0
    arr[inf] = 0
    return arr

In [9]:
df_val = inf_nan_to_zero(df_train.ix[val_idxes])
df_train = inf_nan_to_zero(df_train.drop(df_train.index[val_idxes]))
y_train = y_train.drop(y_train.index[val_idxes])

In [10]:
# Calculate the class weights to simulate the class disribution of testing samples
#
# These numbers and formula is got from here: 
#     https://www.kaggle.com/lystdo/quora-question-pairs/lstm-with-word2vec-embeddings#175198

def get_weights(gt):
    test_set_pos_label_ratio = 0.1746
    training_pos_ratio = sum(gt)/len(gt)

    same_question_ratio = 0 # a training hyper paramter to add testcases like [A,A] 
    random_negative_samples_ratio = 0

    weights = {
        0: (1-test_set_pos_label_ratio) / (1-training_pos_ratio),
        1: test_set_pos_label_ratio/training_pos_ratio
    }
    return weights

weights = get_weights(y_train)
y_train_weights = np.array([weights[0] if y==0 else weights[1] for y in y_train])

weights = get_weights(y_val)
y_val_weights = np.array([weights[0] if y==0 else weights[1] for y in y_val])

In [ ]:
# from sklearn.preprocessing import StandardScaler

# ss = StandardScaler()
# ss.fit(np.vstack([df_train,df_test]))

# df_train = ss.transform(df_train)
# df_val = ss.transform(df_val)
# df_test = ss.transform(df_test)

In [ ]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'

params['eta'] = 0.02
params['max_depth'] = 7
params['subsample'] = 0.6
params['base_score'] = 0.2
 
# params['eta'] = 0.1
# params['max_depth'] = 5
# params['subsample'] = 0.7
# params['colsample_bytree'] = 0.5
# params['colsample_bylevel'] = 0.5
# params['min_child_weight'] = 10

# params['base_score'] = 0.1


d_train = xgb.DMatrix(df_train, label=y_train, weight=y_train_weights)
d_valid = xgb.DMatrix(df_val, label=y_val, weight=y_val_weights)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 100000, watchlist, early_stopping_rounds=100, verbose_eval=50)

/home/nlplab/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0]	train-logloss:0.454963	valid-logloss:0.455608
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[50]	train-logloss:0.257184	valid-logloss:0.26828
[100]	train-logloss:0.207992	valid-logloss:0.221891
[150]	train-logloss:0.190306	valid-logloss:0.205805
[200]	train-logloss:0.182328	valid-logloss:0.198787
[250]	train-logloss:0.177601	valid-logloss:0.194956
[300]	train-logloss:0.173787	valid-logloss:0.192064
[350]	train-logloss:0.170618	valid-logloss:0.189909
[400]	train-logloss:0.167894	valid-logloss:0.188351
[450]	train-logloss:0.165501	valid-logloss:0.186982
[500]	train-logloss:0.163567	valid-logloss:0.186041
[550]	train-logloss:0.161795	valid-logloss:0.185331
[600]	train-logloss:0.16013	valid-logloss:0.184651
[650]	train-logloss:0.158537	valid-logloss:0.184088
[700]	train-logloss:0.157103	valid-logloss:0.183528
[750]	train-logloss:0.155634	valid-logloss:0.183069
[800]	train-logloss:0

In [35]:
len(list(df_train.columns))

201

In [38]:
for c in d_test.feature_names:
    if c not in list(df_train.columns):
        print(c)

In [36]:
len(d_test.feature_names)

201

## Save to model predictions

In [ ]:
test_frames = [read_csv(file, base='features_from_model/test/') for file in feature_files]
df_test = pd.concat(test_frames, axis=1)
df_test = inf_nan_to_zero(df_test)
del test_frames

d_test = xgb.DMatrix(df_test)
del df_test

In [ ]:
output_name = 'xgboost_all_features.csv'

# # process train

# d_test = xgb.DMatrix(df_train)
# pred = bst.predict(d_test)

# sub = pd.DataFrame()
# sub['is_duplicate'] = pred
# sub.to_csv('./model_predictions/train/' + output_name, index=False)

# process test

pred = bst.predict(d_test)

sub = pd.DataFrame()
sub['is_duplicate'] = pred
sub.to_csv('./model_predictions/train/' + output_name, index=False)

## prediction

In [11]:
output_name = 'prediction/xgb_loss0141.csv'

In [13]:
d_test = xgb.DMatrix(df_test)
p_test = bst.predict(d_test)

sub = pd.DataFrame()
sub['test_id'] = np.arange(2345796)
sub['is_duplicate'] = p_test
sub.to_csv(output_name, index=False)